# Double source plane cosmology forecast

In [ ]:
# import standard python modules
import numpy as np
import corner
import copy
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

# import lenstronomy and hierArc modules
from lenstronomy.Cosmo.lens_cosmo import LensCosmo
from hierarc.Sampling.ParamManager.cosmo_param import CosmoParam
from hierarc.Sampling.mcmc_sampling import MCMCSampler

# matplotlib configs
from pylab import rc
rc('axes', linewidth=2)
rc('xtick',labelsize=15)
rc('ytick',labelsize=15)

## Forecast settings

Here we describe the redshift distribution, precision on Einstein radii ratios of an anticipated sample


In [ ]:
from hierarc.Likelihood.LensLikelihood.double_source_plane import beta_double_source_plane

# define a cosmology
cosmology = 'FLCDM'  # Flat LCDM cosmology 
# other options are: "FwCDM", "w0waCDM", "oLCDM"
kwargs_cosmo_true = {'h0': 70, 'om': 0.3}  # cosmological model of the forecast

# create astropy.cosmology instance of input cosmology
cosmo_param = CosmoParam(cosmology=cosmology)
cosmo_true = cosmo_param.cosmo(kwargs_cosmo_true)


# =====================================
# TODO: match settings with publication
# =====================================

# number of double source plane lenses
num_dspl = 87

sigma_beta = 0.05  # relative precision on Einstein radius ratio

def draw_lens():
    """
    draw the likelihood object of a double source plane lens
    """
    z_lens = np.random.uniform(low=0.2, high=0.5) 
    z_source1 = np.random.uniform(low=z_lens+0.2, high=3)
    z_source2 = np.random.uniform(low=z_source1, high=5)
    beta = beta_double_source_plane(z_lens, z_source1, z_source2, cosmo_true)
    beta_measured = beta + np.random.normal(loc=0, scale=sigma_beta * beta)
    kwargs_likelihood = {'z_lens': z_lens, 'z_source_1': z_source1, 'z_source_2': z_source2, 
                         'beta_dspl': beta_measured, 'sigma_beta_dspl': sigma_beta * beta,
                         'likelihood_type': 'DSPL'}
    return kwargs_likelihood


kwargs_dspl_list = []

for i in range(num_dspl):
    kwargs_dspl_list.append(draw_lens())



## hierArc sampling settings

In [ ]:
# sampling settings
n_walkers = 100
n_run = 200
n_burn = 200

cosmology = 'FwCDM'

kwargs_mean_start = {'kwargs_cosmo': {'h0': 70, 'om': 0.3, 'w': -1}}

kwargs_sigma_start = {'kwargs_cosmo': {'h0': 10, 'om': 0.05, 'w': 0.2}}


kwargs_bounds = {'kwargs_lower_cosmo': {'h0': 0, 'om': 0, 'w': -2},
                'kwargs_upper_cosmo': {'h0': 200, 'om': 1, 'w': 0},
                'kwargs_fixed_cosmo': {'h0': kwargs_cosmo_true['h0']}}




# joint options for hierArc sampling
kwargs_sampler = {'cosmology': cosmology, 'kwargs_bounds': kwargs_bounds,
                  'lambda_mst_sampling': False,  
                  'anisotropy_sampling': False,
                  'kappa_ext_sampling': False, 'kappa_ext_distribution': 'NONE', 
                  'alpha_lambda_sampling': False, 'sigma_v_systematics': False,
                  'log_scatter': False, 
                  'custom_prior': None, 'interpolate_cosmo': False,
                  'num_redshift_interp': 100, 'cosmo_fixed': None}



In [ ]:
mcmc_sampler = MCMCSampler(kwargs_dspl_list, **kwargs_sampler)
mcmc_samples, log_prob = mcmc_sampler.mcmc_emcee(n_walkers, n_burn, n_run, kwargs_mean_start, kwargs_sigma_start)

corner.corner(mcmc_samples, show_titles=True, labels=mcmc_sampler.param_names(latex_style=True))
plt.show()